In [ ]:
#匯入所需套件
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd

# 資料匯入與預處理

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**(1) 股票名稱: Acer Incorporated 代碼: 2353.TW**

In [ ]:
#讀取下載下來的股價資料
stock = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2353.TW.csv')
print(len(stock)) #總資料筆數
stock.head(10) #前10筆股價

(2) **呈現前後幾筆股價**

In [ ]:
#刪除有missing value之紀錄
stock = stock.dropna() #只要有一欄位有missing value就刪除
stock.index = range(len(stock)) #重新設定索引
print(len(stock)) #總資料筆數
stock #觀看前後幾筆股價

**(3) 預測股價: 收盤價**

In [ ]:
#決定要預測的股價欄位，例如欲預測收盤價，可輸入'Close'
stock_set = stock['Close'] #預測收盤價

In [ ]:
ref_day = int(5) #ref_day:預測參照天數

*將分割時間點由1/31改為3/31。*

In [ ]:
#分割訓練資料與測試資料
length = len(stock[stock.Date<'2022-03-31'])-ref_day #決定分割時間點，例如以2022-03-31前ref_day天以前之有交易記錄的資料為training dataset
train = stock_set[0:length] #決定資料分割時間點，例如以2022-01-31前ref_day天起為testing dataset
test = stock_set[length:]
train

**(3) 正規化結果**

In [ ]:
from sklearn import preprocessing #使用sklearn的preprocessing的MinMaxScaler進行正規化:(x-min)/(max-min)
#執行正規化
train = train.values.reshape(-1,1) #執行reshape，使其shape為(資料長度,1)
scaler= preprocessing.MinMaxScaler(feature_range=(0, 1)).fit(train)
train_nom = scaler.transform(train) #執行正規化，使得特徵值範圍介於0~1
train_nom[0:9]

In [ ]:
#設定資料之input variable (X) 與target variable (Y)。如以被預測日前ref_day天作為input variables，以被預測日做為target variable
def xy_set(dataset, ref_day):
    predict_day=1 #predict_day:被預測天數
    X = [[0 for i in range(ref_day)] for j in range(dataset.shape[0]-predict_day-ref_day)] #宣告輸入變數空集合
    Y = [0 for j in range(dataset.shape[0]-predict_day-ref_day)] #宣告被預測變數空集合
    for i in range(dataset.shape[0]-predict_day-ref_day):
        X[i] = dataset[i:i+ref_day,0] #前ref_day天資料
        Y[i] = dataset[i+ref_day:i+ref_day+predict_day,0] #第ref_day+1天資料
    return np.array(X),np.array(Y)

In [ ]:
X_train, Y_train = xy_set(train_nom, ref_day) #區分訓練資料之input variables與output variable
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) # 轉成三維，以便符合keras recurrent之input_shape
Y_train = np.squeeze(Y_train)
print('輸入變數維度',X_train.shape)
print('預測變數維度',Y_train.shape)
print('第Y個被預測日之前ref_day天每天正規化股價')
print(X_train[4])
print('第Y個預測日之正規化股價')
print(Y_train[4])

# 模型建立

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM

In [ ]:
model = Sequential() #建立神經網路層

*降低Dropout由0.2降至0.1。*

In [ ]:
model.add(LSTM(units = 16, input_shape = (ref_day,1))) #於RNN層立16個神經元; input_shape(timesteps, features)
model.add(Dropout(0.1))

*「激活函數」改為relu，減少梯度消失。*

In [ ]:
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.1))

In [ ]:
model.add(Dense(units=1, activation='sigmoid'))

**(4) 模型摘要**

In [ ]:
model.summary()

In [ ]:
#model.compile(loss="mean_squared_error", optimizer="adam",metrics=['mean_squared_error']) # 屬迴歸分析，通常使用Mean Squared Error或Mean Absolute Error
model.compile(loss="mean_absolute_error", optimizer="adam",metrics=['mean_absolute_error'])

**(5) 預測模型訓練過程**

*將validation_split由0.2降為0.1。*

In [ ]:
train_history =model.fit(X_train, Y_train,batch_size=32,  
                         epochs=100,verbose=1,validation_split=0.1)

In [ ]:
%pylab inline
#讓圖形直接顯示於jupyter note
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation): #train_history訓練過程；train訓練結果；validation驗證結果
    plt.plot(train_history.history[train]) #描繪訓練結果
    plt.plot(train_history.history[validation]) #描繪驗證結果
    plt.title('Train History') #圖標題
    plt.ylabel(train) #y軸標籤
    plt.xlabel('Epoch') #x軸標籤
    plt.legend(['train', 'validation'], loc='upper left') #設定訓練與驗證之圖例及位置
    plt.show()

**(6) 測試資料之 mean_squared_error 或 mean_absolute_error**

In [ ]:
show_train_history(train_history,'mean_absolute_error','val_mean_absolute_error') #訓練過程繪製成圖

*將分割時間點由1/31改為3/31。*

In [ ]:
#分割訓練資料與測試資料
length = len(stock[stock.Date<'2021-03-31'])-ref_day #決定分割時間點，例如以2021-01-31之前的ref_day天以後之有交易記錄的資料為testing set
test = stock_set[length:]
#執行正規化
test = test.values.reshape(-1,1) #執行reshape，使其shape為(資料長度,1)
test_scaler= preprocessing.MinMaxScaler(feature_range=(0, 1)).fit(test)
test_nom = test_scaler.transform(test) #執行正規化，使得特徵值範圍介於0~1

In [ ]:
X_test, Y_test = xy_set(test_nom, ref_day) #區分訓練資料之input variables與output variable
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1)) # 轉成三維，以便符合keras recurrent之input_shape
Y_test = np.squeeze(Y_test)
print('輸入變數維度',X_test.shape)
print('預測變數維度',Y_test.shape)
print('第Y個被預測日之前ref_day天每天正規化股價')
print(X_test[4])
print('第Y個預測日之正規化股價')
print(Y_test[4])

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=1) #使用model.evaluate進行模型評估，評估後的績效指標會儲存於scores
scores[1] #傳回測試資料的mean_absolute_error

In [ ]:
Y_test_predict=model.predict(X_test) #測試資料預測結果
Y_test_predict_price = test_scaler.inverse_transform(Y_test_predict) # 將正規化預測值轉換為股票價格

In [ ]:
X_test_real, Y_test_real = xy_set(test, ref_day) #取得測試資料之被預測日之真實股票價格

In [ ]:
def show_test_result(Real_data,Prediciton_data):
    plt.plot(Real_data, color="red", label="Real Stock Price") #描繪測試資料真實股價
    plt.plot(Prediciton_data, color="blue", label="Predicted Stock Price") #描繪測試資料預測股價
    plt.title('Stock Prediction')
    plt.ylabel('Price') 
    plt.xlabel('Time') 
    plt.legend()
    plt.show()

**(7) 測試資料預測股價與實際股價之比較(請繪圖)**

In [ ]:
show_test_result(Y_test_real,Y_test_predict_price) #結果若不錯，代表可以使用此模型預測某支股票的股價